In [1]:
from model import LSTMModel
import torch
import sklearn.metrics as metrics

In [2]:
def get_metrics(model_name,model,dataloader):
    model.eval()
    num_class=4
    total_correct=0
    total=0
    tp=[0]*num_class
    fp=[0]*num_class
    fn=[0]*num_class
    tn=[0]*num_class
    dict=[]
    for i in range(num_class):
        dict.append([0]*num_class)
    for i,(x,y) in enumerate(dataloader):
        y_pred=model(x)
        y_pred=torch.argmax(y_pred,dim=1)
        y=torch.argmax(y,dim=1)
        total_correct+=torch.sum(y_pred==y).item()
        total+=len(y)
        for i in range(num_class):
            tp[i]+=torch.sum((y_pred==i)&(y==i)).item()
            fp[i]+=torch.sum((y_pred==i)&(y!=i)).item()
            fn[i]+=torch.sum((y_pred!=i)&(y==i)).item()
            tn[i]+=torch.sum((y_pred!=i)&(y!=i)).item()
        for j in range(len(y)):
            dict[y_pred[j]][y[j]]+=1
    acc=total_correct/total
    accuracy_per_class=[0]*num_class    
    precision=[0]*num_class
    recall=[0]*num_class
    f1=[0]*num_class
    for i in range(num_class):
        accuracy_per_class[i]=tp[i]/(tp[i]+fn[i])
        if tp[i]+fp[i]!=0:
            precision[i]=tp[i]/(tp[i]+fp[i])
        if tp[i]+fn[i]!=0:
            recall[i]=tp[i]/(tp[i]+fn[i])
        if precision[i]+recall[i]!=0:
            f1[i]=2*precision[i]*recall[i]/(precision[i]+recall[i])

    print('Accuracy:',acc)
    fpr=[fp[i]/(fp[i]+tn[i]) for i in range(num_class)]
    fnr=[fn[i]/(fn[i]+tp[i]) for i in range(num_class)]
    print('Classes:',['negative','learning','recall','recognition_familiar'])
    for i in range(num_class):
        print(accuracy_per_class[i],f1[i],recall[i],precision[i],fpr[i],fp[i],fnr[i],fn[i])
    print('Confusion matrix:')
    print(dict)




In [ ]:
for x in ['model2.5/2_16/model53.pt','model2.5/2_32/model23.pt','model2.5/2_64/model30.pt','model2.5/3_16/model63.pt','model2.5/3_32/model29.pt','model2.5/3_64/model26.pt','model2.5/4_10/model29.pt','model2.5/4_16/model51.pt','model2.5/4_32/model48.pt','model2.5/4_64/model64.pt']:
    print(x)
    second=x.split('/')[0]
    second=second[5:]
    val_dataloader=torch.load('val'+second+'_dataloader.pt')
    test_dataloader=torch.load('test'+second+'_dataloader.pt')
    model=torch.load(x)
    get_metrics(x,model,val_dataloader)
    get_metrics(x,model,test_dataloader)

In [ ]:
for x in ['model3/2_16/model45.pt','model3/2_32/model20.pt','model3/2_64/model25.pt','model3/3_16/model39.pt','model3/3_32/model29.pt','model3/3_64/model26.pt','model3/4_10/model30.pt','model3/4_16/model45.pt','model3/4_32/model37.pt','model3/4_64/model66.pt']:
    print(x)
    second=x.split('/')[0]
    second=second[5:]
    val_dataloader=torch.load('val'+second+'_dataloader.pt')
    test_dataloader=torch.load('test'+second+'_dataloader.pt')
    model=torch.load(x)
    get_metrics(x,model,val_dataloader)
    get_metrics(x,model,test_dataloader)

In [8]:
for x in ['model3/3_16/model39.pt']:
    print(x)
    second=x.split('/')[0]
    second=second[5:]
    train_dataloader=torch.load('train'+second+'_dataloader.pt')
    val_dataloader=torch.load('val'+second+'_dataloader.pt')
    test_dataloader=torch.load('test'+second+'_dataloader.pt')
    model=torch.load(x)
    #get_metrics(x,model,train_dataloader)
    get_metrics(x,model,val_dataloader)
    get_metrics(x,model,test_dataloader)

model3/3_16/model39.pt
Accuracy: 0.8468468468468469
Classes: ['negative', 'learning', 'recall', 'recognition_familiar']
0.8537794299876085 0.9107732980832783 0.8537794299876085 0.9759206798866855 0.026729559748427674 17 0.14622057001239158 118
0.8043478260869565 0.8505747126436782 0.8043478260869565 0.9024390243902439 0.02056555269922879 24 0.1956521739130435 54
0.8833333333333333 0.6666666666666666 0.8833333333333333 0.5353535353535354 0.10926365795724466 138 0.11666666666666667 21
0.8444444444444444 0.8128342245989305 0.8444444444444444 0.7835051546391752 0.0332541567695962 42 0.15555555555555556 28
Confusion matrix:
[[689, 2, 14, 1], [2, 222, 3, 19], [112, 18, 159, 8], [4, 34, 4, 152]]
Accuracy: 0.8660455486542443
Classes: ['negative', 'learning', 'recall', 'recognition_familiar']
0.9062286105407256 0.9345332627492501 0.9062286105407256 0.9646630236794171 0.05083857442348008 97 0.09377138945927448 274
0.822463768115942 0.8397040690505548 0.822463768115942 0.8576826196473551 0.028235

In [ ]:
arr=[[757, 12, 4, 6, 1, 0], [57, 339, 10, 3, 7, 0], [3, 2, 31, 15, 52, 4], [10, 5, 21, 23, 37, 4], [5, 7, 91, 62, 144, 19], [1, 0, 2, 2, 2, 0]]
#Plot confusion matrix
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()
#Show values
for i in range(6):
    for j in range(6):
        ax.text(j, i, arr[i][j], ha='center', va='center', color='black')
cax = ax.matshow(arr, cmap='Blues')
fig.colorbar(cax)
#Make bigger plot
fig.set_size_inches(10, 10)
plt.xticks(np.arange(6),['negative','learning','recall_correct','recall_fail','recognition_correct','recognition_fail'])
plt.yticks(np.arange(6),['negative','learning','recall_correct','recall_fail','recognition_correct','recognition_fail'])
plt.xlabel('True label')
plt.ylabel('Predicted label')



In [ ]:
for x in ['model3.5/2_16/model43.pt','model3.5/2_32/model39.pt','model3.5/2_64/model31.pt','model3.5/3_16/model27.pt','model3.5/3_32/model29.pt','model3.5/3_64/model29.pt','model3.5/4_10/model56.pt','model3.5/4_16/model50.pt','model3.5/4_32/model23.pt','model3.5/4_64/model68.pt']:
    print(x)
    second=x.split('/')[0]
    second=second[5:]
    val_dataloader=torch.load('val'+second+'_dataloader.pt')
    test_dataloader=torch.load('test'+second+'_dataloader.pt')
    model=torch.load(x)
    get_metrics(x,model,val_dataloader)
    get_metrics(x,model,test_dataloader)